In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
spay_mutasi = pd.read_csv('Spay Disburse Report 1-7 Mar 24.csv')

In [4]:
spay_mutasi_raw = pd.read_csv('Spay Disburse Report 1-7 Mar 24.csv')

In [5]:
fee_divide = spay_mutasi['Fee (Processing)'] / 2 

In [6]:
spay_mutasi.insert(loc=9, column='Fee/2', value=fee_divide)

In [7]:
spay_gds = pd.read_csv('Spay Disburse GDS 1-7 Mar 24.csv')

In [8]:
spay_gds_raw = pd.read_csv('Spay Disburse GDS 1-7 Mar 24.csv')

In [9]:
print(len(spay_mutasi))
print(len(spay_gds))

173648
173647


In [10]:
spay_mutasi[spay_mutasi['External Reference ID'].duplicated()]

,Merchant Host,Partner Merchant ID,Merchant/Store Name,Merchant Display Name,Transaction Type,Transaction ID,Reference ID,Disbursement Amount,Fee (Processing),Fee/2,Transaction Amount,Create Time,Update Time,Time,Merchant Scope,External Reference ID,Issuer Identifier,Terminal ID,Currency Code


In [11]:
spay_gds[spay_gds['Reference Number'].duplicated()]

,Tx ID,Last Updated,Username,Source Account,Domain,Product Type,Amount,Admin Fee,Admin Fee Invoice,Disburse Cost,Tx Status,disburse_status,Bank Code,Bank Account,Vendor,Reference Number,Tx Ref Number,Vendor Ref ID,money_movement_cost,testing_account


In [12]:
spay_mutasi = spay_mutasi.drop(['Merchant Host','Partner Merchant ID','Merchant/Store Name','Transaction Type','Reference ID','Create Time','Time','Merchant Scope','Issuer Identifier','Terminal ID','Currency Code'],axis = 1)

In [13]:
spay_mutasi = spay_mutasi.add_suffix('_MUTASI')

In [14]:
spay_gds = spay_gds.drop(['Source Account','Domain','Bank Code','Bank Account','Vendor','Tx Ref Number','testing_account','money_movement_cost'],axis = 1)

In [15]:
spay_gds = spay_gds.add_suffix('_GDS')

In [16]:
spay_gds['Last Updated_GDS'] = pd.to_datetime(spay_gds['Last Updated_GDS']).dt.date

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_17786/3341801140.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  spay_gds['Last Updated_GDS'] = pd.to_datetime(spay_gds['Last Updated_GDS']).dt.date


In [17]:
spay_gds.rename(columns={"Last Updated_GDS": "Date"}, inplace=True)

In [18]:
data_compare = pd.merge(spay_mutasi,spay_gds, left_on='External Reference ID_MUTASI', right_on='Reference Number_GDS',how='outer', indicator=True)

In [19]:
data_compare['Match?'] = np.where((data_compare['External Reference ID_MUTASI'] == data_compare['Reference Number_GDS']) & (data_compare['Transaction Amount_MUTASI'] == data_compare['Amount_GDS']) , "Recon", "Unrecon")

In [20]:
data_compare['_merge'] = data_compare['_merge'].cat.rename_categories({'left_only': 'Dash Only', 'right_only': 'GDS Only'})

In [21]:
unrecon = data_compare[data_compare['Match?'] == 'Unrecon']

In [22]:
index_unrecon = data_compare[data_compare['Match?'] == 'Unrecon'].index

In [23]:
index_unrecon

Index([173647], dtype='int64')

In [24]:
data_compare = data_compare.drop(index_unrecon)

In [25]:
data_compare

,Merchant Display Name_MUTASI,Transaction ID_MUTASI,Disbursement Amount_MUTASI,Fee (Processing)_MUTASI,Fee/2_MUTASI,Transaction Amount_MUTASI,Update Time_MUTASI,External Reference ID_MUTASI,Tx ID_GDS,Date,...,Amount_GDS,Admin Fee_GDS,Admin Fee Invoice_GDS,Disburse Cost_GDS,Tx Status_GDS,disburse_status_GDS,Reference Number_GDS,Vendor Ref ID_GDS,_merge,Match?
0,PT Bank OCBC NISP,MWSB2O6JGN2C2AG3IO5S2LNAZ2UIM,500000,600,300.0,500000,3/1/24,20240229175947849571,a6dbac10-f8e7-46d0-813b-6567ed23e4f3,2024-03-01,...,500000.0,666.0,0.0,300.0,SUCCESS,SUCCESS,20240229175947849571,MWSB2O6JGN2C2AG3IO5S2LNAZ2UIM,both,Recon
1,CIMB Niaga,MWSKL5EDOUJCCCRGFGRSCP6O1FBBM,500000,800,400.0,500000,3/1/24,20240229175946236581,65106d9c-e6a6-4c44-8e52-a39deac9785e,2024-03-01,...,500000.0,555.0,0.0,400.0,SUCCESS,SUCCESS,20240229175946236581,MWSKL5EDOUJCCCRGFGRSCP6O1FBBM,both,Recon
2,CIMB Niaga,MWSJ2OLZKFAERBHAQTRZUSDSDKY6M,99000,800,400.0,99000,3/1/24,20240229175942248920,447cfee5-89c4-4255-b28c-102adf243e1d,2024-03-01,...,99000.0,555.0,0.0,400.0,SUCCESS,SUCCESS,20240229175942248920,MWSJ2OLZKFAERBHAQTRZUSDSDKY6M,both,Recon
3,PT Bank OCBC NISP,MWSNUV2XG4IEZHIIADD3YJCDFER3Y,100000,600,300.0,100000,3/1/24,20240229175936913536,d35f402f-da63-4772-b0c5-e87d54ac4a6f,2024-03-01,...,100000.0,666.0,0.0,300.0,SUCCESS,SUCCESS,20240229175936913536,MWSNUV2XG4IEZHIIADD3YJCDFER3Y,both,Recon
4,CIMB Niaga,MWSTKHMMJM5WYANWPZJPQTIH3KUC3,129000,800,400.0,129000,3/1/24,20240229175927237691,de67c277-dd2b-415b-9968-f932746b140f,2024-03-01,...,129000.0,555.0,0.0,400.0,SUCCESS,SUCCESS,20240229175927237691,MWSTKHMMJM5WYANWPZJPQTIH3KUC3,both,Recon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173642,CIMB Niaga,MWSOAWRHYG2KOHKZ4DZH45HUKYIK3,120000,800,400.0,120000,3/7/24,20240307160032719416,8610462e-a675-4be4-a138-0fa85f3e2266,2024-03-07,...,120000.0,555.0,0.0,400.0,SUCCESS,SUCCESS,20240307160032719416,MWSOAWRHYG2KOHKZ4DZH45HUKYIK3,both,Recon
173643,CIMB Niaga,MWSKBBSOPXRFI2Z4C1XDZ2RSRP1ZQ,40000,800,400.0,40000,3/7/24,20240307160031584222,8f8279db-1137-478f-ad0c-97c53493558a,2024-03-07,...,40000.0,555.0,0.0,400.0,SUCCESS,SUCCESS,20240307160031584222,MWSKBBSOPXRFI2Z4C1XDZ2RSRP1ZQ,both,Recon
173644,PT Bank OCBC NISP,MWSIHRVYGLTXM3EF63I6V1141LWWQ,248394,600,300.0,248394,3/7/24,20240307160029504139,6111433a-ffbb-4da5-900f-1c374a3c7515,2024-03-07,...,248394.0,666.0,0.0,300.0,SUCCESS,SUCCESS,20240307160029504139,MWSIHRVYGLTXM3EF63I6V1141LWWQ,both,Recon
173645,PT Bank OCBC NISP,MWSDIWBJ6BF5U2VAWIOE1RR2ON43Y,300000,600,300.0,300000,3/7/24,20240307160013583967,c21acebe-1366-491b-8888-22aa525c17c4,2024-03-07,...,300000.0,666.0,0.0,300.0,SUCCESS,SUCCESS,20240307160013583967,MWSDIWBJ6BF5U2VAWIOE1RR2ON43Y,both,Recon


In [26]:
#groupByUsernametes = data_compare[['External Reference ID_MUTASI','Date','Transaction Amount_MUTASI','Fee/2_MUTASI','Amount_GDS','Reference Number_GDS', 'Disburse Cost_GDS']]

In [27]:
#groupByUsernametes

In [28]:
#groupByUsernametes = groupByUsernametes.groupby(['Date']).agg({'External Reference ID_MUTASI':'count','Transaction Amount_MUTASI' : 'sum', 'Fee/2_MUTASI' :'sum', 'Reference Number_GDS' : 'count','Amount_GDS' : 'sum', 'Disburse Cost_GDS' : 'sum' }, dropna=False)

In [29]:
#groupByUsernametes

In [30]:
#groupByUsername = data_compare[['Username','Product Type','Amount','Disburse Cost','Admin Fee', 'Admin Fee Invoice']]

In [31]:
#groupByUsername = groupByUsername.groupby(['Username','Product Type']).agg({'Username':'count','Amount' : 'sum', 'Disburse Cost' :'sum', 'Admin Fee' : 'sum', 'Admin Fee Invoice' : 'sum' })

In [32]:
#groupByUsername

In [33]:
with pd.ExcelWriter("Spay Disburse 1-7 Mar 24 New Template.xlsx") as writer:
    spay_gds_raw.to_excel(writer, sheet_name="GDS Raw", index=False)
    spay_mutasi_raw.to_excel(writer, sheet_name="Mutasi Report Raw", index=False)
    data_compare.to_excel(writer, sheet_name="Line Per Line Recon", index=False)
    unrecon.to_excel(writer, sheet_name="Unrecon", index=False)
    

In [34]:
print('done')

done


NIC Report (Append)

In [35]:
NIC = spay_mutasi_raw[['Update Time', 'Transaction ID','Transaction Amount','Fee (Processing)']]

In [36]:
NIC.rename(columns={"Update Time": "Date", 'Transaction Amount' : 'TPV', 'Fee (Processing)' : 'MM Cost'}, inplace=True)

/var/folders/sg/v7nz4brs3sq_pn793b_nw89c0000gp/T/ipykernel_17786/3825831630.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NIC.rename(columns={"Update Time": "Date", 'Transaction Amount' : 'TPV', 'Fee (Processing)' : 'MM Cost'}, inplace=True)


In [37]:
NIC = NIC.groupby(['Date']).agg({'Transaction ID':'count','TPV' : 'sum', 'MM Cost' :'sum'})

In [38]:
NIC.rename(columns={"Transaction ID": "#Trx"}, inplace = True)

In [39]:
NIC

,#Trx,TPV,MM Cost
Date,,,
3/1/24,27199,11304100261,20017600
3/2/24,27340,8033111917,19575800
3/3/24,35106,9021705057,25502000
3/4/24,21578,6896995584,15757600
3/5/24,23892,10164849936,17809000
3/6/24,21343,6076745723,16014600
3/7/24,17190,4322399152,12925200


In [40]:
with pd.ExcelWriter("NIC Spay Disburse.xlsx") as writer:
    NIC.to_excel(writer, sheet_name="NIC SPay Disburse", index=True) #notes : using this code only for the first date 

In [41]:
print("done")

done


In [42]:
reader = pd.read_excel('NIC Spay Disburse.xlsx')

In [43]:
len(reader) +1

8

In [44]:
with pd.ExcelWriter('NIC Spay Disburse.xlsx',
                    mode='a', if_sheet_exists = 'overlay') as writer:  
    NIC.to_excel(writer, sheet_name='NIC SPay Disburse', startrow = len(reader) + 1, header=False, index=True)

In [45]:
print('Done')

Done
